In [ ]:
! pip install undetected-chromedriver

In [11]:

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from bs4 import BeautifulSoup

individual_urls = []
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
options = uc.ChromeOptions()
# options.add_argument("--headless")
options.add_argument(f"user-agent={my_user_agent}")

driver = uc.Chrome(service=Service('helpers/chromedriver-win64/chromedriver.exe'), options=options)
driver.set_window_size(1920, 1080) 
# Get women shoes
driver.get("https://www.skechers.com/women/shoes/")
soup = BeautifulSoup(driver.page_source, 'html.parser')
div_ = soup.find_all('div', class_="c-product-tile__image-container")
for div in div_:
    individual_urls.append("https://www.skechers.com" + div.find('a')['href'])
# Get men shoes
driver.get("https://www.skechers.com/men/shoes/")
soup = BeautifulSoup(driver.page_source, 'html.parser')
div_ = soup.find_all('div', class_="c-product-tile__image-container")
for div in div_:
    individual_urls.append("https://www.skechers.com" + div.find('a')['href'])
details = []
for index, url in enumerate(individual_urls):
    print(f"trying {index} - {url}")
    link_detail = {"link": url, "brand": "sketchers"}
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    link_detail["shoe_name"] =  soup.select_one('.product-name').text.strip().replace("\n", "").replace("  ","")
    link_detail["category"] = soup.select_one('.c-product-details__label').text.strip().replace("\n", "").replace("  ","")
    link_detail["style_or_product_code"] = soup.select_one('.c-product-details__style').text.strip().replace("\n", "").replace("  ","")
    link_detail["reviews"] = soup.select_one('.pr-snippet-review-count').text.strip().replace("\n", "").replace("  ","").replace('NoReviews',"0")
    available_sizes = [size.text.strip() for size in soup.select('.c-product-attributes__item__value--size')]
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = [color.get("aria-label").strip().replace("Select Color ","") for color in soup.select('.js-color-attr-selector')]
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    star_list = soup.select('.pr-star-v4')
    stars = 0
    for star in star_list:
        count = star.get("class")[-1].split("-")[-2]
        if count.isdigit():
            stars += int(count)/100
    link_detail["stars"] = stars
    link_detail["price"] = soup.select_one('.sales .value').text.strip()
    link_detail["quality"], link_detail["comfort"], link_detail["width"] = 0,0,0
    details.append(link_detail)

driver.quit()



trying 0 - https://www.skechers.com/premium-leather-slip-ins-snoop-one---og/186001_WHT.html
trying 1 - https://www.skechers.com/skechers-slip-ins-bobs-sport-squad-chaos/117500_ROS.html
trying 2 - https://www.skechers.com/skechers-slip-ins-bobs-sport-squad-chaos/117504_LAV.html
trying 3 - https://www.skechers.com/skechers-slip-ins-ultra-flex-3.0---brilliant/149710_PERI.html
trying 4 - https://www.skechers.com/skechers-slip-ins-ultra-flex-3.0/150174_WHT.html
trying 5 - https://www.skechers.com/skechers-slip-ins-ultra-flex-3.0---new-energy/150182_BKNT.html
trying 6 - https://www.skechers.com/skechers-slip-ins-ultra-flex-3.0---smooth-step/149709_MVE.html
trying 7 - https://www.skechers.com/skechers-slip-ins-ultra-flex-3.0---cozy-streak/149708_SLT.html
trying 8 - https://www.skechers.com/martha-stewart-x-skechers-slip-ins-ultra-flex-3/149808_OLV.html
trying 9 - https://www.skechers.com/skechers-slip-ins-summits---dazzling-haze/149937_GYMT.html
trying 10 - https://www.skechers.com/skechers-s

In [12]:
import pandas as pd
df1_columns = ["shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["reviews", "size", "comfort", "quality", "width", "stars"]
sketchers_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in details)
sketchers_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in details)
sketchers_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in details)

print(sketchers_df1)
print("\n")

print(sketchers_df2)
print("\n")

print(sketchers_df3)
print("\n")

                                            shoe_name category  \
0             Premium Leather Slip-ins Snoop One - OG  Women's   
1           Skechers Slip-ins: Bobs Sport Squad Chaos  Women's   
2           Skechers Slip-ins: Bobs Sport Squad Chaos  Women's   
3       Skechers Slip-ins: Ultra Flex 3.0 - Brilliant  Women's   
4                   Skechers Slip-ins: Ultra Flex 3.0  Women's   
5      Skechers Slip-ins: Ultra Flex 3.0 - New Energy  Women's   
6     Skechers Slip-ins: Ultra Flex 3.0 - Smooth Step  Women's   
7     Skechers Slip-ins: Ultra Flex 3.0 - Cozy Streak  Women's   
8    Martha Stewart x Skechers Slip-ins: Ultra Flex 3  Women's   
9          Skechers Slip-ins: Summits - Dazzling Haze  Women's   
10         Skechers Slip-ins: Summits - Diamond Dream  Women's   
11   Martha Stewart x Skechers Slip-ins: Ultra Flex 3  Women's   
12   Martha Stewart Skechers Slip-ins: Ultra Flex 3.0  Women's   
13           Skechers Slip-ins: GO WALK Flex - Relish  Women's   
14        